In [ ]:
# Reads in photometry from different sources, normalizes them, and puts them
# onto a BJD time scale

# Created 2021 Dec. 28 by E.S.

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib qt

In [2]:
## USER INPUTS

photometry_source_file = "sv_eri_ptf_prepolished.txt"
bjd_file = "sv_eri_ptf_bjd.csv" # generated as-we-go
writeout_polished_file = "sv_eri_ptf_polished.txt"

In [3]:
stem_super = "/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/"
stem_moore_macadam = stem_super + "polished_macadam_moore_photometry/"
stem_public_sources = stem_super + "all_photometry_program_stars/polished/"
bjd_sources = stem_super + "all_photometry_program_stars/bjds/"

In [5]:
# read in photometry

df = pd.read_csv(stem_public_sources + photometry_source_file)

In [6]:
# if there's a bunch of too-old stuff, remove it
#df = df.where(np.logical_and(df["JD"]>2456474-400,df["JD"]<2456474+400)).dropna(how="all").reset_index()
#df = df.where(df["JD"]<=2459238.567486600).dropna(how="all").reset_index()

In [7]:
df.keys()

Index(['obsmjd', 'mag_autocorr', 'magerr_auto', 'oid', 'ra', 'dec', 'fid',
       'transient_flag', 'astrometricrms', 'nobs', 'ngoodobs', 'nbestobs',
       'referencemag', 'referencemagerr', 'medianmag', 'bestmedianmag',
       'meanmag', 'bestmeanmag', 'magrms', 'bestmagrms', 'sid', 's_ra',
       's_dec', 'ptffield', 'ccdid', 'hmjd', 'nid', 'mag_auto', 'limitmag',
       'absphotzp', 'fwhmsex', 'fwhm_image', 'qaflags', 'flags', 'photcalflag',
       'goodflag', 'pid', 'dist', 'angle'],
      dtype='object')

In [10]:
# write out only JD, for conversion

#df["JD"] = np.add(df["obsmjd"],2400000.5)
df["JD"].to_csv("junk_jds.csv", header=False, index=False)

# at this point, do conversion step manually online

In [ ]:
# FYI: location of Wichita (approx. center of U.S.)

'''
lat_wichita = 37.688889
lon_wichita = -97.336111
'''

# read bjds back in and tack onto the dataframe

In [11]:
intermed_bjd = pd.read_csv(bjd_sources + bjd_file, names=["BJD"])
print(bjd_sources + bjd_file)

df["BJD"] = intermed_bjd.values

#df = df.join(intermed_bjd["BJD"])

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/bjds/sv_eri_ptf_bjd.csv


In [12]:
len(df)

50

In [13]:
df

,obsmjd,mag_autocorr,magerr_auto,oid,ra,dec,fid,transient_flag,astrometricrms,nobs,...,fwhm_image,qaflags,flags,photcalflag,goodflag,pid,dist,angle,JD,BJD
0,56267.328357,10.971,0.273,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,19.185,0,29491204,0,1,14758674,1.381514,135.566205,2.456268e+06,2.456268e+06
1,56239.215257,10.855,0.024,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,26.783,0,29491204,0,1,14580152,1.381514,135.566205,2.456240e+06,2.456240e+06
2,56239.231027,10.879,0.024,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,26.879,0,29491204,0,1,14580164,1.381514,135.566205,2.456240e+06,2.456240e+06
3,56239.273737,10.952,0.024,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,26.600,0,29491204,0,1,14580193,1.381514,135.566205,2.456240e+06,2.456240e+06
4,56251.161207,10.734,0.021,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,28.518,0,29491204,0,1,14655851,1.381514,135.566205,2.456252e+06,2.456252e+06
5,56251.157567,10.719,0.021,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,28.248,0,29491204,0,1,14655852,1.381514,135.566205,2.456252e+06,2.456252e+06
6,56253.146377,11.119,0.022,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,23.759,0,29491204,0,1,14675024,1.381514,135.566205,2.456254e+06,2.456254e+06
7,56253.175687,11.036,0.022,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,24.520,0,29491204,0,1,14675042,1.381514,135.566205,2.456254e+06,2.456254e+06
8,56253.235617,10.840,0.022,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,27.401,0,29491204,0,1,14675078,1.381514,135.566205,2.456254e+06,2.456254e+06
9,56204.362307,11.227,0.269,20832040004119,47.967064,-11.354174,2,0,NaN,62,...,4.122,0,29491204,0,1,14314634,1.381514,135.566205,2.456205e+06,2.456205e+06


In [18]:
# in case of weird naming conventions
'''
df["Magnitude"] = df["mag_autocorr"]
df["Uncertainty"] = df["magerr_auto"]
'''

# look at the data, choose what we want

In [19]:
plt.clf()
plt.scatter(df["BJD"],
            df["Magnitude"])
plt.show()

In [14]:
plt.clf()
plt.scatter(df["BJD"],
            df["mag_autocorr"])
plt.show()

In [13]:
plt.clf()
plt.scatter(df["JD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="k")
plt.scatter(df["BJD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="b")
plt.show()

In [18]:
plt.clf()
plt.scatter(df["BJD"].where(df["Observer Code"] == "SAH"),
            df["Magnitude"].where(df["Observer Code"] == "SAH"), color="k")
plt.show()

In [13]:
df.keys()

Index(['JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

In [12]:
print(np.min(df["JD"]))
print(np.max(df["JD"]))

2439144.571
2459546.39706


In [11]:
len(df)

30529

In [15]:
df["Band"].value_counts()

KeyError: 'Band'

In [16]:
df["Observer Code"].value_counts()

KeyError: 'Observer Code'

In [17]:
# choose photometric subset

df_subset_photo = df
#df_subset_photo = df.where(np.logical_and(df["BJD"] > 2452456,df["Band"] == "V"))
#df_subset_photo = df.where(np.logical_or(df["BJD"] < 2455109,df["BJD"] > 2455535))
#df_subset_photo = df.where(df["Band"] == "V")
#df_subset = df[["JD", "sdf"]]

In [17]:
# check to confirm
plt.clf()
plt.scatter(df_subset_photo["BJD"],df_subset_photo["Magnitude"])
plt.show()

In [13]:
np.max(df_subset_photo["BJD"])

2456809.3022545683

In [14]:
df_subset_photo.keys()

Index(['index', 'JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

# feature scale the photometry

### wait! scaling of error bars won't be the same, since an offset is being subtracted off

In [29]:
'''
scaler = MinMaxScaler() # min-max mapped to 0-to-1
df_subset_photo[['mag_scaled']] = scaler.fit_transform(df_subset_photo[['Magnitude']])
# df_subset_photo[['error_scaled']] = scaler.transform(df_subset_photo[['Uncertainty']])
'''

In [21]:
'''
plt.scatter(df_subset_photo["BJD"],df_subset_photo["mag_scaled"])
plt.show()
'''

# write out as 'polished' photometry

In [ ]:
# cols BJD, mag, error

In [20]:
# remove rows of all NaNs

df_subset_photo_dropna = df_subset_photo.dropna(axis=0, how='all')

In [21]:
# write out only the relevant cols

df_subset_photo_dropna.to_csv(stem_public_sources + writeout_polished_file, 
                              columns=["BJD","Magnitude","Uncertainty"], index=False)
print(stem_public_sources + writeout_polished_file)

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/polished/sv_eri_ptf_polished.txt
